In [77]:
# !pip install pubchempy
# !pip install pyarrow

In [2]:
import json
import pickle
import numpy as np
from ocpmodels.datasets import SinglePointLmdbDataset 
import os
import ase.io
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, molecule, surface
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs, SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
from pprint import pprint
import pubchempy as pcp
from transformers import AutoTokenizer, AutoModel, pipeline
import pandas as pd
import ray
import torch
import random

2023-08-19 13:34:12,698	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
seed = 42  # or any other number
# Set the seed for numpy, torch, Python's random module, and sklearn's train_test_split
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# # load dataset
# print('- load dataset')
# src = 'ocp/data/is2re/all/train/data.lmdb'
# dataset = SinglePointLmdbDataset({"src": src})
   
# # generate ads energy mappings
# print('- create energy dataset')
# keys = []
# energies = []
# for data in dataset:
#     keys.append('random' + str(data.sid))
#     energies.append(data.y_relaxed)
    
# energy_data = dict(zip(keys, energies))

# # Open a file and use dump()
# with open('mapping_energies_train.pkl', 'wb') as file:
      
#     # A new file will be created
#     pickle.dump(energy_data, file)

# print('done.')

# # load dataset
# print('- load dataset')
# src = 'ocp/data/is2re/all/val_id/data.lmdb'
# dataset = SinglePointLmdbDataset({"src": src})
   
# # generate ads energy mappings
# print('- create energy validation dataset')
# keys = []
# energies = []
# for data in dataset:
#     keys.append('random' + str(data.sid))
#     energies.append(data.y_relaxed)
    
# energy_data = dict(zip(keys, energies))

# # Open a file and use dump()
# with open('mapping_energies_validation.pkl', 'wb') as file:
      
#     # A new file will be created
#     pickle.dump(energy_data, file)

# print('done.')

In [5]:
# load mappings

print('- load data mapping')
with open('oc20_data_mapping.pkl', 'rb') as f:
    mapping_data = pickle.load(f)

print('- load adslab_slab mapping')
with open('mapping_adslab_slab.pkl', 'rb') as f:
    mapping_adslab_slab = pickle.load(f)

print('- load energy mapping - train')
with open('mapping_energies_train.pkl', 'rb') as f:
    energy_data_train = pickle.load(f)

print('- load energy mapping - validation')
with open('mapping_energies_validation.pkl', 'rb') as f:
    energy_data_validation = pickle.load(f)

- load data mapping
- load adslab_slab mapping
- load energy mapping - train
- load energy mapping - validation


In [6]:
# Merge dictionaries
print('- merging all energy data')
energy_data = {**energy_data_train, **energy_data_validation}

common_keys = list(set(energy_data.keys()) & set(mapping_data.keys()))

print(len(common_keys))

- merging all energy data
485271


In [7]:
list_data = []
for ck in common_keys[:]:
    # print(energy_data[ck])
    # print(mapping_data[ck])
    data_extra = {
        'adslab_slab_key': ck,
        'energy': energy_data[ck]
    }
    data = {**mapping_data[ck], **data_extra}
    list_data.append(data)

# with open('datasets/ocp_reactions.pickle', 'wb') as f:
#     pickle.dump(list_data, f)

with open('datasets/ocp_reactions.pickle', 'rb') as f:
    loaded_list_data = pickle.load(f)

In [8]:
# Convert list to DataFrame
df = pd.DataFrame(loaded_list_data)

print(df.shape)

# Sort by energy
df = df.sort_values(by='energy')

# Drop duplicates based on bulk_mpid and miller_index, keeping the entry with the smallest energy
df = df.drop_duplicates(subset=['bulk_mpid', 'miller_index', 'ads_symbols'], keep='first')

print(df.shape)

(485271, 13)
(446885, 13)


In [9]:
# Convert back to list of dictionaries (if required)
loaded_list_data = df.to_dict(orient='records')

print(len(loaded_list_data))

# with open('datasets/ocp_reactions_info.pickle', 'wb') as f:
#     pickle.dump(loaded_list_data, f)

with open('datasets/ocp_reactions_info.pickle', 'rb') as f:
    loaded_list_data = pickle.load(f)

print(len(loaded_list_data))

446885
446885


In [10]:
## reaction info to data

for lld in loaded_list_data[:]:
    ads_symbols = lld['ads_symbols']
    pkey = ads_symbols.replace('*', '')
    lld['pkey'] = pkey

pprint(loaded_list_data[-1])

list_pkey =  []
for data in loaded_list_data:
    pkey = data['pkey']
    list_pkey.append(pkey)

list_pkey = list(set(list_pkey))

len(list_pkey)

{'ads_id': 67,
 'ads_symbols': '*NO2NO2',
 'adslab_slab_key': 'random1645311',
 'adsorption_site': ((6.08, 5.63, 18.81),),
 'anomaly': 0,
 'bulk_id': 2106,
 'bulk_mpid': 'mp-11329',
 'bulk_symbols': 'P4W2',
 'class': 2,
 'energy': 9.999100859999942,
 'miller_index': (1, 1, 1),
 'pkey': 'NO2NO2',
 'shift': 0.022,
 'top': True}


68

In [11]:
# def name_to_smiles(name):
#     try:
#         compounds = pcp.get_compounds(name, namespace='formula')
#         if compounds:
#             return compounds[0].isomeric_smiles
#         else:
#             print(f"No compound found for {name}")
#             return None
#     except Exception as e:
#         print(f"Error for {name}: {e}")
#         return None

# list_pkey_smiles = []
# for pkey in list_pkey:
#     smiles = name_to_smiles(pkey)
#     # print(pkey, smiles)
#     list_pkey_smiles.append((pkey, smiles))    

# d_pkey_vs_smiles = {}
# for (pkey, smiles) in list_pkey_smiles:
#     d_pkey_vs_smiles[pkey] = smiles
    
# 'ONN(CH3)2': 'ONN(C)(C)',

In [12]:
d_pkey_vs_smiles = {
    'CH2CO': 'C=C=O',
    'COHCHO': 'C(=O)C=O',
    'OH2': 'O',
    'CH2O': 'C=O',
    'COCHO': 'C(=C=O)[O-]',
    'CHOH': 'C=O',
    'ONN(CH3)2': 'ONN(C)(C)',
    'OHCH2CH3': 'CCO',
    'NONH': 'N=N[O-]',
    'NO2': 'N(=O)[O-]',
    'CCHO': 'C#C[O-]',
    'OHCH3': 'CO',
    'NH': '[NH]',
    'COHCOH': 'C(=O)C=O',
    'OCH2CH3': 'CC[O-]',
    'NNO': '[N-]=[N+]=O',
    'CHCH2OH': 'C1CO1',
    'CHOCH2OH': 'CC(=O)O',
    'CHCOH': 'C=C=O',
    'NO': '[N]=O',
    'CHOHCHOH': 'CC(=O)O',
    'CH2OH': 'C[O-]',
    'NNH': '[NH+]#N',
    'CCH2': 'C#C',
    'OCH3': 'C[O-]',
    'CCH3': 'C=[CH]',
    'CH2CH3': 'C[CH2+]',
    'CHOHCH2': 'C1CO1',
    'COHCH2OH': 'CC(=O)O',
    'OCH2CHOH': 'CC(=O)O',
    'CN': '[C-]#N',
    'CHOHCH3': 'CC[O-]',
    'CCH2OH': 'C[C]=O',
    'CHOCHO': 'C(=O)C=O',
    'CHCH': 'C#C',
    'CH4': 'C',
    'COCH2O': 'C(=O)C=O',
    'CHOCHOH': 'CC(=O)[O-]',
    'CCH': 'C#[C-]',
    'CHOHCH2OH': 'C([CH]O)O',
    'N2': 'N#N',
    'ONNH2': 'NN=O',
    'N': '[N]',
    'OCHCH3': 'C1CO1',
    'C': '[C]',
    'ONH': 'N=O',
    'CHCO': 'C#C[O-]',
    'CCO': 'C1#CO1',
    'CH2CH2OH': 'CC[O-]',
    'NO3': '[N+](=O)([O-])[O-]',
    'O': '[O]',
    'NO2NO2': '[N+](=O)([N+](=O)[O-])[O-]',
    'CH2': '[CH2]',
    'CHCH2': 'C=[CH]',
    'OH': '[OH-]',
    'CC': '[C-]#[C+]',
    'NHNH': 'N=N',
    'H': '[H+]',
    'COHCHOH': 'CC(=O)[O-]',
    'CH3': '[CH3+]',
    'NH3': 'N',
    'OHNNCH3': 'C(=O)(N)N',
    'CHCHOH': 'C[C]=O',
    'COCH3': 'C[C]=O',
    'CCHOH': 'C=C=O',
    'OHNH2': 'NO',
    'COHCH3': 'C1CO1',
    'CHCHO': 'C=C=O'
}

In [13]:
## pretrained model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/chEMBL26_smiles_v2")
model = AutoModel.from_pretrained("mrm8488/chEMBL26_smiles_v2")
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer, device=-1) ## device= (0 for GPU, -1 for CPU)

list_pkey= list(d_pkey_vs_smiles.keys())
list_smiles= list(d_pkey_vs_smiles.values())
list_emb_1 = fe(list_smiles)

list_emb_2 = []
for emb_1 in list_emb_1:
    emb_2 = np.mean(np.array(emb_1[0]), axis=0)
    list_emb_2.append(emb_2)
arr_emb = np.array(list_emb_2)
print(arr_emb.shape)

d_pkey_vs_pdesc = {}
for i, pkey in enumerate(list_pkey):
    pdesc = arr_emb[i]
    d_pkey_vs_pdesc[pkey] = pdesc    

Some weights of the model checkpoint at mrm8488/chEMBL26_smiles_v2 were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(68, 768)


In [15]:
## Save dictionary to pickle file
# with open('datasets/d_pkey_vs_pdesc.pickle', 'wb') as file:
#     pickle.dump(d_pkey_vs_pdesc, file)

In [16]:
# Load dictionary from pickle file
with open('datasets/d_pkey_vs_pdesc.pickle', 'rb') as file:
    d_pkey_vs_pdesc = pickle.load(file)

In [17]:
def find_key(miller_index, bulk_symbol, ads_symbol):
    matching_keys = []
    
    for key, values in mapping_data.items():
        if (values['miller_index'] == miller_index and
            values['bulk_symbols'] == bulk_symbol and
            values['ads_symbols'] == ads_symbol):
            matching_keys.append(key)
            
    return matching_keys

def get_ads_energy(miller_index, bulk_symbol, ads_symbol):
    my_keys = find_key(miller_index, bulk_symbol, ads_symbol)
    all_keys = energy_data.keys()
    
    energies = []
    for key in my_keys:
        print('MAPDATA: ', key)
        print(json.dumps(mapping_data[key], indent=4))
        if key in all_keys:
            energies.append(energy_data[key])
            print('ENERGY: ', energy_data[key])
        else:
            print('NO ENERGY')
        print(' ')
            
    return energies

# miller_index = (1, 1, 1)
# bulk_symbol = 'Pd'
# ads_symbol = '*OH'

# keys = find_key(miller_index, bulk_symbol, ads_symbol)
# print(keys)
# energies = get_ads_energy(miller_index, bulk_symbol, ads_symbol)
# print(energies) 

In [18]:
list_bulk_facet = []
for data in loaded_list_data:
    bulk = data['bulk_mpid']
    facet = data['miller_index']
    list_bulk_facet.append((bulk, facet))

list_bulk_facet = list(set(list_bulk_facet))

In [19]:
len(list_bulk_facet)

93244

In [20]:
d_bulk_facet_vs_sdesc = {}
for bulk_facet in list_bulk_facet:
    d_bulk_facet_vs_sdesc[bulk_facet] = []

In [31]:
## Initialize Ray.
# ray.init(ignore_reinit_error=True, num_cpus=50)

In [64]:
m = MPRester('Yct0KDbJbqMLWluZEovkwrLXh2VRHXbc')
list_adsorbate = ['H', 'O', 'C', 'CO', 'OH', 'N2', 'CO2', 'NH3', 'CH4'] ## 'H2S' 

config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
checkpoint_path = "dimenetpp_all.pt"

# Initialize the calculator
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)

/home/mchowdh5/.local/lib/python3.10/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [33]:
##=======================================================================================================

In [34]:
# @ray.remote
# def main_execution_remote(start_index, end_index, list_bulk_facet, list_adsorbate, m, calc):
#     result = {}
#     # list_bulk_facet = ray.get(list_bulk_facet_id)
#     # m = ray.get(m_id)
#     # list_adsorbate = ray.get(list_adsorbate_id)
#     # calc = ray.get(calc_id)            
#     for ibulkfacet in range(start_index, end_index):    
#         try:
#             bulk, facet = list_bulk_facet[ibulkfacet]
#             structure = m.get_structure_by_material_id(bulk)
#             slabgen = SlabGenerator(structure, facet, 10, 10)
#             slabs = slabgen.get_slabs()
#             slab = slabs[0] ## todo: what if no slab? what if multiple slab?

#             list_energy = []
#             for ads_symbol in list_adsorbate:
#                 adslab = AseAtomsAdaptor().get_atoms(slab)
#                 adslab.calc = calc        
#                 slab_energy = adslab.get_potential_energy()

#                 adsorbate = molecule(ads_symbol)
#                 add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
#                 tags = np.zeros(len(adslab))
#                 tags[18:27] = 1
#                 tags[27:] = 2
#                 adslab.set_tags(tags)
#                 cons = FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
#                 adslab.set_constraint(cons)
#                 adslab.center(vacuum=13.0, axis=2)
#                 adslab.set_pbc(True)
#                 ads_energy = adslab.get_potential_energy()
#                 energy = ads_energy - slab_energy ## todo: is this also applicable here?   
#                 list_energy.append(energy)

#             result[(bulk, facet)] = list_energy
#         except Exception as e:
#             pass 
#     return result

In [35]:
# slim, elim, glim = 0, 93244, 1000 ## 0, 93244, 1000
# futures = []
# for start_index in range(slim, elim, glim):
#     end_index = start_index + glim
#     if end_index > elim:
#         end_index = elim

#     # Fire off a remote execution.
#     future = main_execution_remote.remote(start_index, end_index, list_bulk_facet, list_adsorbate, m, calc)
#     futures.append(future)

# # Retrieve results.
# results = ray.get(futures)

In [36]:
# merged_dict = {}
# for d in results:
#     merged_dict.update(d)

# with open('datasets/d_bulk_facet_vs_sdesc_0to93244.pickle', 'wb') as f:
#     pickle.dump(merged_dict, f)

In [37]:
##=======================================================================================================

In [39]:
with open('datasets/d_bulk_facet_vs_sdesc_0to10k.pickle', 'rb') as f:
    d0 = pickle.load(f)
with open('datasets/d_bulk_facet_vs_sdesc_10kto20k.pickle', 'rb') as f:
    d1 = pickle.load(f)
with open('datasets/d_bulk_facet_vs_sdesc_20kto30k.pickle', 'rb') as f:
    d2 = pickle.load(f)
with open('datasets/d_bulk_facet_vs_sdesc_30kto60k.pickle', 'rb') as f:
    d3 = pickle.load(f)
with open('datasets/d_bulk_facet_vs_sdesc_60kto93244.pickle', 'rb') as f:
    d4 = pickle.load(f)

In [40]:
print(f"Number of items in d0: {len(d0)}")
print(f"Number of items in d1: {len(d1)}")
print(f"Number of items in d2: {len(d2)}")
print(f"Number of items in d3: {len(d3)}")
print(f"Number of items in d4: {len(d4)}")

Number of items in d0: 9070
Number of items in d1: 9990
Number of items in d2: 9993
Number of items in d3: 29179
Number of items in d4: 33066


In [41]:
d_bulk_facet_vs_sdesc = {**d0, **d1, **d2, **d3, **d4} 

In [42]:
len(d_bulk_facet_vs_sdesc)

91298

In [44]:
# with open('datasets/d_bulk_facet_vs_sdesc.pickle', 'wb') as file:
#     pickle.dump(d_bulk_facet_vs_sdesc, file)

In [45]:
with open('datasets/d_bulk_facet_vs_sdesc.pickle', 'rb') as file:
    d_bulk_facet_vs_sdesc = pickle.load(file)

In [50]:
len(d_bulk_facet_vs_sdesc)

91298

In [51]:
len(d_pkey_vs_pdesc)

68

In [52]:
len(loaded_list_data)

446885

In [53]:
pprint(loaded_list_data[-1])

{'ads_id': 67,
 'ads_symbols': '*NO2NO2',
 'adslab_slab_key': 'random1645311',
 'adsorption_site': ((6.08, 5.63, 18.81),),
 'anomaly': 0,
 'bulk_id': 2106,
 'bulk_mpid': 'mp-11329',
 'bulk_symbols': 'P4W2',
 'class': 2,
 'energy': 9.999100859999942,
 'miller_index': (1, 1, 1),
 'pkey': 'NO2NO2',
 'shift': 0.022,
 'top': True}


In [54]:
c0, c1 = 0, 0
for idata, data in enumerate(loaded_list_data):
    pkey = data['pkey']
    skey = (data['bulk_mpid'], data['miller_index'])
    pdesc = list(d_pkey_vs_pdesc[pkey])
    try: 
        sdesc = list(d_bulk_facet_vs_sdesc[skey])
        c1+=1
    except Exception as e:
        sdesc = []
        c0+=1
    data['pdesc'] = pdesc
    data['sdesc'] = sdesc    
    if idata%100000==0:
        print(idata)

0
100000
200000
300000
400000


In [55]:
print(c0, c1, c0+c1)

9104 437781 446885


In [57]:
pprint(len(list(loaded_list_data[-1].keys())))

16


In [58]:
df = pd.DataFrame(loaded_list_data)

In [59]:
df.head(2)

,bulk_id,ads_id,bulk_mpid,bulk_symbols,ads_symbols,miller_index,shift,top,adsorption_site,class,anomaly,adslab_slab_key,energy,pkey,pdesc,sdesc
0,2085,29,mp-976273,Hf6Ge4,*COCH2O,"(2, 1, 0)",0.022,True,"((9.3, 4.58, 26.89),)",1,1,random2024607,-9.992999,COCH2O,"[1.4773670434951782, 0.012636385030216642, -0....","[0.008257746696472168, 1.0352611541748047, 1.2..."
1,10724,20,mp-1247259,Ca6Rh2N6,*CHCO,"(1, 0, 0)",0.312,True,"((10.54, 1.25, 22.82),)",2,0,random1171399,-9.982733,CHCO,"[1.6455507054924965, 0.9016518630087376, -0.22...","[0.005488276481628418, 0.16277754306793213, 0...."


In [60]:
print(df.shape)

(446885, 16)


In [61]:
df = df[df['sdesc'].str.len() > 0]

In [62]:
print(df.shape)

(437781, 16)


In [65]:
list_scols = ['s' + str(i) for i in range(len(list_adsorbate))]

In [66]:
df[list_scols] = pd.DataFrame(df.sdesc.tolist(), index= df.index)

In [67]:
print(df.shape)

(437781, 25)


In [68]:
list_pcols = ['p' + str(i) for i in range(int(arr_emb[0].shape[0]))]

In [69]:
try:
    df[list_pcols] = pd.DataFrame(df.pdesc.tolist(), index= df.index)
except Exception as e:
    print(e)

In [78]:
print(df.shape)

(437781, 793)


In [79]:
df.head(2)

,bulk_id,ads_id,bulk_mpid,bulk_symbols,ads_symbols,miller_index,shift,top,adsorption_site,class,...,p758,p759,p760,p761,p762,p763,p764,p765,p766,p767
0,2085,29,mp-976273,Hf6Ge4,*COCH2O,"(2, 1, 0)",0.022,True,"((9.3, 4.58, 26.89),)",1,...,-0.698056,0.273536,0.742442,0.618160,0.486411,-0.934350,0.204278,0.679431,0.341144,-0.852883
1,10724,20,mp-1247259,Ca6Rh2N6,*CHCO,"(1, 0, 0)",0.312,True,"((10.54, 1.25, 22.82),)",2,...,-0.625508,-0.456795,0.304594,1.134157,-0.394405,-1.171003,-0.648138,-0.021550,-1.443121,0.004959


In [80]:
# df.to_parquet('datasets/ocp_reactions_info_df.parquet')

In [81]:
df = pd.read_parquet('datasets/ocp_reactions_info_df.parquet')

In [82]:
print(df.shape)

(437781, 793)


In [83]:
df.rename(columns={'energy': 'nre'}, inplace=True)

In [84]:
df = df[list_scols + list_pcols + ['nre']]

In [85]:
print(df.shape)

(437781, 778)


In [86]:
df.head(2)

,s0,s1,s2,s3,s4,s5,s6,s7,s8,p0,...,p759,p760,p761,p762,p763,p764,p765,p766,p767,nre
0,0.008258,1.035261,1.294881,1.552592,1.783934,1.012493,2.079694,0.436319,-1.365212,1.477367,...,0.273536,0.742442,0.618160,0.486411,-0.934350,0.204278,0.679431,0.341144,-0.852883,-9.992999
1,0.005488,0.162778,0.328713,1.856944,1.908573,1.835093,1.429667,0.374538,-1.153396,1.645551,...,-0.456795,0.304594,1.134157,-0.394405,-1.171003,-0.648138,-0.021550,-1.443121,0.004959,-9.982733


In [88]:
# df.to_parquet('datasets/ocp_reactions_info_df_dataset.parquet')

In [89]:
df = pd.read_parquet('datasets/ocp_reactions_info_df_dataset.parquet')

In [90]:
print(df.shape)

(437781, 778)
